<a href="https://colab.research.google.com/github/iiCellxx/CSST-101-ALMARIO/blob/main/LSTM_Practice_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
!pip install tensorflow


In [137]:
import pandas as pd

# Load fashion data from CSV
fashion_df = pd.read_csv('/content/CSV File/fashion_chatbot_data.csv')

# Load greetings data from CSV
greetings_df = pd.read_csv('/content/CSV File/Greetings.csv')



# Check if all datasets have the same columns (patterns, responses)
print(fashion_df.columns)
print(greetings_df.columns)


# Align columns if necessary
# Assuming that all datasets should have 'patterns' and 'responses' columns
greetings_df.columns = fashion_df.columns  # Align columns with the fashion dataset


# Combine all datasets (fashion, greetings, and conversation)
combined_df = pd.concat([fashion_df, greetings_df], ignore_index=True)

# Save the combined dataset as a CSV
combined_csv_path = '/content/combined_chatbot_data.csv'
combined_df.to_csv(combined_csv_path, index=False)

# Show the combined dataset
combined_df.head()


Index(['patterns', 'responses'], dtype='object')
Index(['patterns', 'responses'], dtype='object')


,patterns,responses
0,What is your favorite color?,"My favorite color is black, very stylish."
1,Tell me about trendy styles.,Trendy styles right now include oversized jack...
2,What is the best outfit for summer?,"For summer, I recommend light, breathable fabr..."
3,Can you recommend a casual look?,"For a casual look, try a comfy t-shirt and jeans."
4,What is the latest fashion trend?,The latest fashion trends include pastel color...


In [138]:
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Download punkt tokenizer
nltk.download('punkt')

# Tokenize the combined patterns
tokenizer = Tokenizer()
tokenizer.fit_on_texts(combined_df['patterns'])
X = tokenizer.texts_to_sequences(combined_df['patterns'])

# Padding the sequences to ensure uniform input length
X_pad = pad_sequences(X, padding='post')

# Encode the responses (dummy encoding for simplicity)
import numpy as np
y = np.array([i for i in range(len(combined_df['responses']))])  # Dummy encoding for simplicity

# Show tokenized input and padded sequences
X_pad[:5], y[:5]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


(array([[ 1,  4, 35, 36, 16,  0,  0,  0,  0],
        [23, 18, 24, 37, 38,  0,  0,  0,  0],
        [ 1,  4,  8, 11,  9,  7, 25,  0,  0],
        [10, 13, 26,  3, 39, 12,  0,  0,  0],
        [ 1,  4,  8, 40, 41, 42,  0,  0,  0]], dtype=int32),
 array([0, 1, 2, 3, 4]))

In [130]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=32, input_length=X_pad.shape[1]))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(combined_df['responses']), activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Summary of the model
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_15 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_16 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [131]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Set up early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
checkpoint_callback = ModelCheckpoint(
    '/content/fashion_greetings_chatbot_model_checkpoint.keras',  # File path for checkpoint
    save_best_only=True,  # Save only the best model (based on validation loss)
    monitor='loss',  # Monitor loss during training
    verbose=1  # Print messages when saving checkpoints
)

# Train the model with both early stopping and checkpointing
model.fit(X_pad, y, epochs=100, batch_size=32, verbose=1, callbacks=[early_stopping, checkpoint_callback])

# Save the final trained model
model.save('/content/fashion_greetings_chatbot_model.keras')


Epoch 1/100
1/2 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0312 - loss: 3.7851
Epoch 1: loss improved from inf to 3.78525, saving model to /content/fashion_greetings_chatbot_model_checkpoint.keras
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - accuracy: 0.0256 - loss: 3.7852
Epoch 2/100
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.0312 - loss: 3.7830
Epoch 2: loss improved from 3.78525 to 3.78291, saving model to /content/fashion_greetings_chatbot_model_checkpoint.keras
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.0256 - loss: 3.7829
Epoch 3/100
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.0312 - loss: 3.7811
Epoch 3: loss improved from 3.78291 to 3.78176, saving model to /content/fashion_greetings_chatbot_model_checkpoint.keras
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.0256 - loss: 3.7816
Epoch 4/100
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.0625 - loss: 3.7811
Epoch 4: loss improved from 3.78176 to 3.78075, saving model to /content/fashion_greetin

In [126]:
# Function to predict responses
def get_response(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=X_pad.shape[1], padding='post')
    pred = model.predict(padded)
    response_idx = np.argmax(pred)
    return combined_df['responses'][response_idx]


In [139]:
# Chatbot loop
print("Hello! Ask me about fashion or greetings. Type 'exit' to end the conversation.")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        print("Goodbye!")
        break
    response = get_response(user_input)
    print("Chatbot:", response)


Hello! Ask me about fashion or greetings. Type 'exit' to end the conversation.
You: Yo
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Chatbot: Yo Mga Ka Fashion
You: Black Outfits?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Chatbot: Athleisure is a mix of athletic wear and casual pieces, perfect for comfort.
You: black bro
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Chatbot: Black Outfits Is The Best.
You: summer black outfit?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Chatbot: Not much, but I'd love to chat!
You: Trendy 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Chatbot: Good morning! Mga Ka Fashion
You: summer outfit?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Chatbot: I'm doing great, thanks for asking!
You: summer
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Chatbot: Greetings! How may I assist you?
You: e about trendy st
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Chatbot: Good morning! Mga Ka Fashion
You: What is your favorite color?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Chatbot: My favorite color is black, very stylish.
You: